# Sistema de Relatório para Escola Bíblica Dominical

## O que este sistema Faz?

Este Sistema gera várias planilhas do excel automaticamente e que podem substituir o livro de relatório físico. Basicamente o sistema cria um Diretório nomeado **Relatorios_EBD** e dentro deste diretório gera diversos diretórios organizados por ano, mês e planilhas para cada domingo. 

## Instale as bibliotecas necessárias com o comando abaixo:

In [ ]:
pip install -r requirements.txt



## Cuidados a serem tomados

Se você rodou o script de criação de diretórios e planilhas para o ano de 2025 e começou a trabalhar nela, não peça para criar 2025 novamente pois os  arquivos serão sobrescritos.

### Abaixo está o script que gera os diretórios e planilhas a serem preenchidas todos os domingos

In [ ]:
import os
import calendar
from datetime import datetime
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
from openpyxl.utils import get_column_letter
from openpyxl.formatting.rule import CellIsRule

# Configurações gerais
print("Não digite planilhas existentes, senão serão substituídas.")
inicio = int(input("Digite o ano inicial: "))
fim = int(input("Digite o ano final: "))
ANOS = range(inicio, fim+1)
CLASSES = [
    "Diretoria", "Abraão", "Samuel", "Ana", "Débora", "Venc. em Cristo",
    "Miriã", "Lírio dos Vales", "Gideões", "Cam. p/ o Céu",
    "Rosa de Saron", "Sold. de Cristo", "Querubins"
]

COLUNAS = ["Classes", "Matriculados", "Ausentes", "Presentes", "Visitantes", "Total", "Bíblias", "Revistas", "Ofertas", "% de Presença"]
MESES_PT = {
    1: "janeiro", 2: "fevereiro", 3: "março", 4: "abril",
    5: "maio", 6: "junho", 7: "julho", 8: "agosto",
    9: "setembro", 10: "outubro", 11: "novembro", 12: "dezembro"
}

# Estilos
header_font = Font(bold=True)
header_fill = PatternFill("solid", fgColor="C0C0C0")
center_align = Alignment(horizontal="center")
fill_zebra = PatternFill("solid", fgColor="F2F2F2")
border = Border(
    left=Side(style='thin'), right=Side(style='thin'),
    top=Side(style='thin'), bottom=Side(style='thin')
)

def domingos_do_mes(ano, mes):
    dias = calendar.monthrange(ano, mes)[1]
    return [day for day in range(1, dias+1) if datetime(ano, mes, day).weekday() == 6]

def cria_planilha_para_domingo(caminho, data):
    nome_arquivo = data.strftime('%d') + f"_{MESES_PT[data.month]}_{data.year}.xlsx"
    wb = Workbook()
    ws = wb.active
    ws.title = "Presença"

    # Título mesclado
    ws.merge_cells("A1:J1")
    ws["A1"] = f"Relatório da Escola Bíblica Dominical em {data.strftime('%d/%m/%Y')}"
    ws["A1"].font = Font(bold=True, size=14)
    ws["A1"].alignment = center_align

    # Cabeçalhos
    for i, coluna in enumerate(COLUNAS, 1):
        celula = ws.cell(row=2, column=i, value=coluna)
        celula.font = header_font
        celula.fill = header_fill
        # Alinhar verticalmente para certas colunas
        vertical_cols = ["Matriculados", "Ausentes", "Presentes", "Visitantes", "Total", "Bíblias", "Revistas", "Ofertas", "% de Presença"]
        if coluna in vertical_cols:
            celula.alignment = Alignment(horizontal="center", vertical="center", textRotation=90)
        else:
            celula.alignment = center_align

        celula.border = border

    # Preenchimento das classes
    for i, classe in enumerate(CLASSES, start=3):
        ws.cell(row=i, column=1, value=classe)
        ws[f"D{i}"] = f"=B{i}-C{i}"  # Presentes
        ws[f"F{i}"] = f"=D{i}+E{i}"  # Total
        ws[f"J{i}"] = f"=IF(B{i}>0,D{i}/B{i},0)"
        ws[f"I{i}"].number_format = 'R$ #,##0.00' # Valor monetário
        ws[f"J{i}"].number_format = '0.00%'
        
        # Zebra striping
        if (i - 3) % 2 == 1:
            for j in range(1, 11):
                ws.cell(row=i, column=j).fill = fill_zebra

        # Aplicar bordas
        for j in range(1, 11):
            ws.cell(row=i, column=j).border = border

    # Linha de total geral
    linha_total = len(CLASSES) + 3
    ws.cell(row=linha_total, column=1, value="Total Geral")
    for col in range(2, 10):
        letra = get_column_letter(col)
        ws.cell(row=linha_total, column=col, value=f"=SUM({letra}3:{letra}{linha_total-1})")
        ws.cell(row=linha_total, column=col).font = Font(bold=True)
        ws.cell(row=linha_total, column=col).alignment = center_align
        ws.cell(row=linha_total, column=col).border = border

    # % de presença total geral
    ws[f"J{linha_total}"] = f"=IF(B{linha_total}>0,D{linha_total}/B{linha_total},0)"
    ws[f"J{linha_total}"].number_format = '0.00%'
    ws[f"J{linha_total}"].font = Font(bold=True)
    ws[f"J{linha_total}"].alignment = center_align
    ws[f"J{linha_total}"].border = border

    # Formatação condicional nas classes
    ws[f"I{linha_total}"].number_format = 'R$ #,##0.00' # Valor monetário
    ws.conditional_formatting.add(
        f"J3:J{linha_total-1}",
        CellIsRule(operator='greaterThanOrEqual', formula=['0.9'], fill=PatternFill(start_color='C6EFCE', end_color='C6EFCE', fill_type='solid'))
    )
    ws.conditional_formatting.add(
        f"J3:J{linha_total-1}",
        CellIsRule(operator='lessThan', formula=['0.7'], fill=PatternFill(start_color='FFC7CE', end_color='FFC7CE', fill_type='solid'))
    )

    # Ajuste automático de largura das colunas
    for col_idx in range(1, ws.max_column + 1):
        col_letter = get_column_letter(col_idx)
        max_length = max(
            len(str(ws.cell(row=row, column=col_idx).value)) if ws.cell(row=row, column=col_idx).value else 0
            for row in range(1, ws.max_row + 1)
        )
        ws.column_dimensions[col_letter].width = max_length + 2

    wb.save(os.path.join(caminho, nome_arquivo))

def criar_estrutura_ebd(base_dir="Relatorios_EBD"):
    os.makedirs(base_dir, exist_ok=True)
    for ano in ANOS:
        ano_dir = os.path.join(base_dir, str(ano))
        os.makedirs(ano_dir, exist_ok=True)
        for mes in range(1, 13):
            nome_mes = MESES_PT[mes]
            mes_dir = os.path.join(ano_dir, nome_mes)
            os.makedirs(mes_dir, exist_ok=True)
            for dia in domingos_do_mes(ano, mes):
                data_domingo = datetime(ano, mes, dia)
                cria_planilha_para_domingo(mes_dir, data_domingo)

# Executar tudo
criar_estrutura_ebd()


# Geração de relatórios trimestrais e anual

Após o preenchimento das planilhas e ao fim do trimestre, semestre ou ano rode o script abaixo: 



In [ ]:
import os
import openpyxl
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, PatternFill
from openpyxl.utils import get_column_letter
from openpyxl.formatting.rule import CellIsRule
from math import isclose

BASE_DIR = "Relatorios_EBD"
CLASSES = [
    "Diretoria", "Abraão", "Samuel", "Ana", "Débora", "Venc. em Cristo",
    "Miriã", "Lírio dos Vales", "Gideões", "Cam. p/ o Céu",
    "Rosa de Saron", "Sold. de Cristo", "Querubins"
]
COLUNAS_RELEVANTES = {
    "Matriculados": 2, "Ausentes": 3, "Presentes": 4,
    "Visitantes": 5, "Total": 6, "Bíblias": 7,
    "Revistas": 8, "Ofertas": 9, "% de Presença": 10
}
TRIMESTRES = {
    "1º Trimestre": ["janeiro", "fevereiro", "março"],
    "2º Trimestre": ["abril", "maio", "junho"],
    "3º Trimestre": ["julho", "agosto", "setembro"],
    "4º Trimestre": ["outubro", "novembro", "dezembro"]
}

# Estilos
bold_center = Font(bold=True)
center_align = Alignment(horizontal="center")
rotated_align = Alignment(horizontal="center", vertical="center", textRotation=90)
currency_format = 'R$ #,##0.00'
percent_format = '0.00%'

# Utilitários
def inicializa_linha_totais():
    return {col: 0 for col in COLUNAS_RELEVANTES}

def somar_linha(origem, destino):
    for k in COLUNAS_RELEVANTES:
        destino[k] += origem[k]

def obter_dados_por_classe(arquivo):
    wb = openpyxl.load_workbook(arquivo, data_only=True)
    ws = wb.active
    dados = {classe: inicializa_linha_totais() for classe in CLASSES}
    for row in ws.iter_rows(min_row=3, max_row=3+len(CLASSES)-1):
        nome_classe = row[0].value
        if nome_classe not in dados:
            continue
        for k, idx in COLUNAS_RELEVANTES.items():
            celula = row[idx-1].value
            valor = float(celula) if celula not in (None, '') else 0
            dados[nome_classe][k] += valor
    return dados

def dividir_e_arredondar(valor, divisor):
    if divisor == 0:
        return 0
    if isinstance(valor, float) and isclose(valor % 1, 0.5, abs_tol=1e-6):
        return int(round(valor))  # evitar erro de arredondamento do Python
    return int(round(valor / divisor))
def criar_relatorio(nome, dados, caminho_destino, contagem_domingos):
    wb = Workbook()
    ws = wb.active
    ws.title = "Relatório"

    # Título principal da planilha
    titulo = f"Relatório do {nome.replace('.xlsx', '')} de {os.path.basename(caminho_destino)} da EBD"
    ws.merge_cells(start_row=1, start_column=1, end_row=1, end_column=len(COLUNAS_RELEVANTES)+1)
    celula_titulo = ws.cell(row=1, column=1, value=titulo)
    celula_titulo.font = Font(bold=True, size=14)
    celula_titulo.alignment = Alignment(horizontal="center")

    # Cabeçalho na linha 2
    cabecalho = ["Classe"] + list(COLUNAS_RELEVANTES.keys())
    for col, texto in enumerate(cabecalho, 1):
        cel = ws.cell(row=2, column=col, value=texto)
        cel.font = bold_center
        if texto == "Classe":
            cel.alignment = center_align
        else:
            cel.alignment = rotated_align

    # Preencher dados por classe a partir da linha 3
    for i, classe in enumerate(CLASSES, start=3):
        ws.cell(row=i, column=1, value=classe)
        for j, k in enumerate(COLUNAS_RELEVANTES.keys(), start=2):
            valor = dados[classe][k]
            cel = ws.cell(row=i, column=j)
            if k == "Ofertas":
                cel.value = valor
                cel.number_format = currency_format
            elif "%" in k:
                media = valor / contagem_domingos if contagem_domingos > 0 else 0
                cel.value = media
                cel.number_format = percent_format
            else:
                cel.value = dividir_e_arredondar(valor, contagem_domingos)
            cel.alignment = center_align

    # Linha de total geral (última linha + 1)
    linha_total = len(CLASSES) + 3
    ws.cell(row=linha_total, column=1, value="Total Geral").font = bold_center
    soma_matriculados = 0
    soma_presentes = 0

    for j, k in enumerate(COLUNAS_RELEVANTES.keys(), start=2):
        total = sum(dados[classe][k] for classe in CLASSES)
        cel = ws.cell(row=linha_total, column=j)
        if k == "Ofertas":
            cel.value = total
            cel.number_format = currency_format
        elif k == "% de Presença":
            soma_matriculados = sum(dados[classe]["Matriculados"] for classe in CLASSES)
            soma_presentes = sum(dados[classe]["Presentes"] for classe in CLASSES)
            porcentagem = soma_presentes / soma_matriculados if soma_matriculados > 0 else 0
            cel.value = porcentagem
            cel.number_format = percent_format
        else:
            cel.value = dividir_e_arredondar(total, contagem_domingos)
        cel.font = bold_center
        cel.alignment = center_align

    # Formatação condicional
    faixa = f"J3:J{linha_total - 1}"
    ws.conditional_formatting.add(
        faixa,
        CellIsRule(operator='greaterThanOrEqual', formula=['0.8'], fill=PatternFill(start_color='C6EFCE', end_color='C6EFCE', fill_type='solid'))
    )
    ws.conditional_formatting.add(
        faixa,
        CellIsRule(operator='lessThan', formula=['0.6'], fill=PatternFill(start_color='FFC7CE', end_color='FFC7CE', fill_type='solid'))
    )

    # Ajuste automático da largura das colunas
    for col in ws.columns:
        max_length = 0
        col_letter = get_column_letter(col[0].column)
        for cell in col:
            try:
                if cell.value:
                    max_length = max(max_length, len(str(cell.value)))
            except:
                pass
        ajuste = min(max_length + 2, 25)
        ws.column_dimensions[col_letter].width = ajuste

    wb.save(os.path.join(caminho_destino, nome))


def gerar_relatorios_anuais_e_trimestrais(ano_escolhido):
    dados_ano = {classe: inicializa_linha_totais() for classe in CLASSES}
    planilhas_ano = 0
    ano_dir = os.path.join(BASE_DIR, str(ano_escolhido))

    for trimestre, meses in TRIMESTRES.items():
        dados_trimestre = {classe: inicializa_linha_totais() for classe in CLASSES}
        contagem_trimestre = 0

        for mes in meses:
            mes_dir = os.path.join(ano_dir, mes)
            if not os.path.exists(mes_dir):
                continue
            for arquivo in os.listdir(mes_dir):
                if arquivo.endswith(".xlsx"):
                    caminho = os.path.join(mes_dir, arquivo)
                    dados = obter_dados_por_classe(caminho)
                    for classe in CLASSES:
                        somar_linha(dados[classe], dados_trimestre[classe])
                        somar_linha(dados[classe], dados_ano[classe])
                    contagem_trimestre += 1
                    planilhas_ano += 1

        criar_relatorio(f"{trimestre}.xlsx", dados_trimestre, ano_dir, contagem_trimestre)

    criar_relatorio("Relatório_Anual.xlsx", dados_ano, ano_dir, planilhas_ano)

# Execução
try:
    ano_input = int(input("Digite o ano para o qual deseja gerar os relatórios (ex: 2024): "))
    gerar_relatorios_anuais_e_trimestrais(ano_input)
    print(f"\n✅ Relatórios gerados com sucesso para o ano {ano_input}!")
except ValueError:
    print("❌ Ano inválido. Digite um número inteiro.")
except Exception as e:
    print(f"❌ Erro inesperado: {e}")
